In [1]:
import numpy as np

# 신경망 복습

## 완전계층구조 - forward만

In [2]:
import numpy as np

w1 = np.random.randn(2,4)
b1 = np.random.randn(4)
x = np.random.randn(10,2)

h = np.matmul(x,W1) +b1

비선형을 위한 활성화 함수

In [9]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [8]:
w1 = np.random.randn(2,4)
b1 = np.random.randn(4)
x = np.random.randn(10,2)

w2 = np.random.randn(4,3)
b2 = np.random.rand(3)

h = np.matmul(x,W1) +b1
a = sigmoid(h)
h2 = np.matmul(a,w2)


이렇게 나온 결과는 단순한 점수score, 클래스별로 확률을 적용하기 위해서는 출력층에 softmax등을 사용해야함.

## 클래스로 구현하기  
각 클래스에는 forward와 backward로 구성.  
초기화로 입력 받는 파라미터들을 params로 선언.

### 구현을 위한 클래스 정의  
시그모이드와 fully connected

In [23]:
import numpy as np

class Sigmoid():
    def __init__(self):
        self.params = []
    def forward(self, x):
        return 1/(1+np.exp(-x))
    
class Affine:
    def __init__(self,W,b):
        self.params=[W,b]
    def forward(self, x):
        W, b = self.params
        out = np.matmul(x,W) + b
        return out

In [26]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, ouput_size):
        
        W1 = np.random.randn(input_size, hidden_size)
        b1 = np.random.randn(hidden_size)
        W2 = np.random.randn(hidden_size, ouput_size)
        b2 = np.random.randn(ouput_size)
        
        self.layers=[
            Affine(W1,b1),
            Sigmoid(),
            Affine(W2,b2)
        ]
        
        self.params = []
        for layer in self.params:
            self.params += layers.params
        
    def predict(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x


In [27]:
x= np.random.randn(10,2)
model = TwoLayerNet(2, 4, 3)
s = model.predict(x)

### 학습을 위한 loss function

# 계산 그래프

## Repeat 노드  & Sum 노드
쉽게 생각하면 한줄의 백터가 있을 때, 해당 백터(로우)를 여러개 반복해서 붙이는것.  
역전파의 계산에서는 해당 노드는 sum을 해준다.  
두 노드는 서로 반대로 작동한다고 생각.


In [31]:
import numpy as np

D, N = 8, 7

x = np.random.randn(1,D)
y = np.repeat(x, N, axis=0)

dy = np.random.randn(N,D)
dx = np.sum(dy,axis=0, keepdims=True)


## matmul 노드  
dL/dX = dL/dY * W^T

In [33]:
class MatMul:
    def __init__(self, W):
        self.params = [W]
        self.grad = [np.zeros_like(W)]
        self.x =None
        
    def forward(self, x):
        W, = self.params
        out = np.matmul(x, W)
        self.x = x
        return out
        
    def backward(self, dout):
        W, = self.params
        dx = np.matmul(dout, W.T)
        dW = np.matmul(self.x.T, dout)
        
        ## numpy deep copy.
        # 완전히 같지는 않지만 C언어에서 포인터의 느낌. 생략기호를 넣으면 값이 완전히 바뀌고,
        # 일반적으로 하듯이 대입을 하면 메모리를 가르키는 위치만 변경.
        self.grads[0][...] = dW
        return dx
    

## sigmoid 노드  
위에서 구현한것을 다시 사용.  
시그모이드는 미분하면 y(1-x)가 됨.
역전파를 전달할떄는 이 수치에 전달받은 미분값을 곱한다.


In [35]:
class Sigmoid():
    def __init__(self):
        self.params = []
        self.grads = []
        self.out = None
        
    def forward(self, x):
        out  = 1/(1+np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * (1 - self.out) * self.out
        return dx

## Affine 계층

In [37]:
class Affine:
    def __init__(self,W,b):
        self.params=[W,b]
        self.grads = [np.zeros_like(W), np.zeros_like(b)]
        self.x = None
        
    def forward(self, x):
        W, b = self.params
        out = np.matmul(x,W) + b
        return out
    
    def bacward(self, dout):
        W, b = self.params
        dx = np.matmul(dout, W.T)
        dw = np.matmul(self.x.T, dout)
        db = np.sum(dout, axis=0)
        
        self.grads[0][...] = dw
        self.grads[1][...] = db
        
        return dx

## softmax with loss 계층  


## 가중치 갱신

In [38]:
class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr
        
    def update(self, parmas, grads):
        for i in range(len(params)):
            params[i] -= self.lr * grads[i]

# 실제 문제 풀기

# 계산을 고속화하기 위한 방법들  
1. 비트 정밀도  
-넘파이는 float64를 기본으로 사용하고 있음. 이를 32비트로 할경우 계산이나 메모리측면에서 이득을 볼수 있음.  

2. 쿠파이  
-GPU좋은것은 요즘 많이 다 아실것이라 생각. 
-numpy를 GPU용으로 사용할수 있는게 cupy!